In [3]:
from typing import List, Any, Dict, Union, Optional, Tuple

from contextlib import contextmanager
import random
import os
import tempfile
import autogen
import openai
import json
import ast

from unittest.mock import Mock
from pathlib import Path
import subprocess
import platform
from dotenv import load_dotenv

from faststream_gen._code_generator.helper import write_file_contents, set_cwd

In [4]:
load_dotenv()

AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')

api_key_sweeden = os.getenv("AZURE_OPENAI_API_KEY_SWEEDEN")
api_base_sweeden = "https://airt-openai-sweden.openai.azure.com/"

api_key_canada = os.getenv("AZURE_OPENAI_API_KEY_CANADA")
api_base_canada = "https://airt-openai-canada.openai.azure.com/"

In [5]:
openai.api_type = "azure"
    
openai.api_version = "2023-07-01-preview"

CONFIG_LIST = [
    {
        "model": "gpt-4",
        "api_key": api_key_sweeden,
        "api_base": api_base_sweeden,
        "api_type": openai.api_type,
        "api_version": openai.api_version,
        "engine": "airt-gpt4"
    },{
        "model": "gpt-4",
        "api_key": api_key_canada,
        "api_base": api_base_canada,
        "api_type": openai.api_type,
        "api_version": openai.api_version,
        "engine": "airt-canada-gpt4"

    },
#     {
#         "model": "gpt-3.5-turbo-16k",
#         "api_key": api_key_sweeden,
#         "api_base": api_base_sweeden,
#         "api_type": openai.api_type,
#         "api_version": openai.api_version,
#         "engine": "airt-gpt35-turbo-16k",
#     }
]

assert openai.api_key is not None, "Please set your AZURE_OPENAI_API_KEY environment variable."

In [3]:
conversation_input = {
  "plan": "1. Define a function named is_palindrome that takes a string as an argument.\n2. Inside the function, convert the string to lowercase using the lower() method.\n3. Remove all spaces and punctuation from the string. We can use the replace() method to remove spaces and the translate() method along with maketrans() to remove punctuation.\n4. Compare the processed string with its reverse. If they are the same, the string is a palindrome. We can get the reverse of a string in Python using slicing with a step of -1.\n5. Return True if the string is a palindrome and False otherwise.\n6. Include a brief description of what the function does at the beginning of the function.\n7. Document the input parameter, specifying its type and what it represents.\n8. Document the return value, specifying its type and what it represents.\n9. Include comments in the code where necessary to explain complex parts of the code.",
  "roles": ["python_dev_gpt", "doc_writer_gpt", "code_executor"]
}
conversation_input

{'plan': '1. Define a function named is_palindrome that takes a string as an argument.\n2. Inside the function, convert the string to lowercase using the lower() method.\n3. Remove all spaces and punctuation from the string. We can use the replace() method to remove spaces and the translate() method along with maketrans() to remove punctuation.\n4. Compare the processed string with its reverse. If they are the same, the string is a palindrome. We can get the reverse of a string in Python using slicing with a step of -1.\n5. Return True if the string is a palindrome and False otherwise.\n6. Include a brief description of what the function does at the beginning of the function.\n7. Document the input parameter, specifying its type and what it represents.\n8. Document the return value, specifying its type and what it represents.\n9. Include comments in the code where necessary to explain complex parts of the code.',
 'roles': ['python_dev_gpt', 'doc_writer_gpt', 'code_executor']}

In [4]:
DIR_NAME = None

def list_files(directory_path: str):
    return "\n".join([f"{str(file)}\n" for file in list(Path(directory_path).iterdir())])

def write_to_file(filename: str, content: str):
    global DIR_NAME
    
    file = DIR_NAME / filename
    file.write_text(content)
    return f"The file was successfully written inside the {filename}"

def read_file(filename: str):
    global DIR_NAME
    
    file = DIR_NAME / filename
    return file.read_text()

# list_files = Mock()
# list_files.side_effect = ["__init__.py, very_important_notes.txt", "__init__.py, very_important_notes.txt, generated_code.py"]

# write_to_file = Mock()
# write_to_file.side_effect = ["The file was written inside the at the wanted location"] * 5

# read_file = Mock()
# read_file.side_effect = ["Please make sure that the palindrome code is saved in the my_palindrome.py", "The file was successfuly read"]

In [43]:
def execute_command(command: List[str]) -> Optional[Tuple[str, str]]:
    global DIR_NAME
    
    if isinstance(command, str):
        command = ast.literal_eval(command)
        
    print(command)
        
    with set_cwd(str(DIR_NAME)):
        # nosemgrep: python.lang.security.audit.subprocess-shell-true.subprocess-shell-true
        p = subprocess.run( # nosec: B602, B603, B607 subprocess call - check for execution of untrusted input.
            command,
            stderr=subprocess.PIPE,
            stdout=subprocess.PIPE,
            shell=True if platform.system() == "Windows" else False,
        )
        
        return f"{str(p.stderr.decode('utf-8'))}\n{str(p.stdout.decode('utf-8'))}"

In [6]:
with tempfile.TemporaryDirectory() as tmpdirname:
    DIR_NAME = Path(tmpdirname)
    
    file = (DIR_NAME / "code.py")
    write_to_file(str(file), "import os")
    print(execute_command(f"['python3', '{str(file)}']"))
    
    print(execute_command(["pytest"]))





============================= test session starts ==============================
platform linux -- Python 3.11.4, pytest-7.4.2, pluggy-1.3.0
rootdir: /tmp/tmp3pm3qnpt
plugins: anyio-3.7.1, asyncio-0.21.1
asyncio: mode=Mode.STRICT
collected 0 items

============================ no tests ran in 0.00s =============================



In [7]:
list_files_config = {
    "name": "list_files",
    "description": "List all the files in the directory",
    "parameters": {
        "type": "object",
        "properties": {
            "directory_path": {
                "type": "string",
                "description": "The path to the directory",
            },
        },
        "required": ["directory_path"],
    },
}

write_to_file_config = {
    "name": "write_to_file",
    "description": "Writes the content into a file",
    "parameters": {
        "type": "object",
        "properties": {
            "filename": {
                "type": "string",
                "description": "The name of the file",
            },
            "content": {
                "type": "string",
                "description": "The content which neeeds to be written into the file",
            }
        },
        "required": ["filename", "content"],
    },
}

read_file_config = {
    "name": "read_file",
    "description": "Reads the content of the file",
    "parameters": {
        "type": "object",
        "properties": {
            "filename": {
                "type": "string",
                "description": "The name of the file",
            },
        },
        "required": ["filename"],
    },
}

ask_for_additional_info_config = {
    "name": "ask_for_additional_info",
    "description": "Ask your supervisor (a person outside of your team) for additional information",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "Question for the supervisor",
            },
        },
        "required": ["question"],
    },
}

execute_command_config = {
    "name": "execute_command",
    "description": "Command which needs to be executed",
    "parameters": {
        "type": "object",
        "properties": {
            "command": {
                "type": "string",
                "description": "Command which needs to be executed e.g. '['pytest']'",
            },
        },
        "required": ["command"],
    },
}

llm_config = {
    "config_list": CONFIG_LIST,
    "seed": 42,
    "temperature": 0.2,
    "functions": [list_files_config, write_to_file_config, read_file_config, ask_for_additional_info_config, execute_command_config]
}


ask_for_additional_info = Mock()
ask_for_additional_info.side_effect = ["I'm not sure", "Do what ever you think is the best solution"] * 5


function_map = {
    "list_files": list_files,
    "write_to_file": write_to_file,
    "read_file": read_file,
    "ask_for_additional_info": ask_for_additional_info,
    "execute_command": execute_command
}

def is_termination_msg(x: str) -> bool:
    return "content" in x and x["content"] is not None and x["content"].rstrip().endswith("TERMINATE")

def create_member(name: str, work_dir: str) -> autogen.Agent:
        name = name.lower().replace(" ", "_")
        system_message = f"""You are {name}
        
Your task is to chat with other team mambers and try to solve the given task.
Do NOT try to finish the task until other team members give their opinion.
        """
        print(f"{system_message}\n{'*'*100}")
        
        return autogen.AssistantAgent(
            name=name,
            llm_config=llm_config,
            system_message=system_message,
            is_termination_msg=is_termination_msg,
            code_execution_config={"work_dir": work_dir},
            function_map=function_map
        )


In [8]:
create_member("developer", "/tmp")

You are developer
        
Your task is to chat with other team mambers and try to solve the given task.
Do NOT try to finish the task until other team members give their opinion.
        
****************************************************************************************************


In [20]:
def create_groupchat(roles: List[str], work_dir: str):
    members = []
    for role in roles:
        members.append(create_member(role, work_dir))

    groupchat = autogen.GroupChat(agents=members, messages=[], max_round=50)
    manager = autogen.GroupChatManager(
        groupchat=groupchat,
        llm_config=llm_config,
        is_termination_msg=is_termination_msg,
    )
    return members, manager

In [21]:
plan = conversation_input["plan"]
roles = conversation_input["roles"]
members, manager = create_groupchat(roles, "/tmp")

You are python_dev_gpt
        
Your task is to chat with other team mambers and try to solve the given task.
Do NOT try to finish the task until other team members give their opinion.
        
****************************************************************************************************
You are doc_writer_gpt
        
Your task is to chat with other team mambers and try to solve the given task.
Do NOT try to finish the task until other team members give their opinion.
        
****************************************************************************************************
You are code_executor
        
Your task is to chat with other team mambers and try to solve the given task.
Do NOT try to finish the task until other team members give their opinion.
        
****************************************************************************************************


In [22]:
def create_message(plan: str, members: List[autogen.Agent], code_directory: str):
    roles = ", ".join([str(member.name) for member in members])
    
    init_message = f"""
You are a team dedicated for implementing the following task in Python:
\n{plan}


The team is consisting of the following roles: {roles}.


Play to your strengths as an LLM and pursue simple strategies with no legal complications.


## Guidelines
1. Each code block must be successfuly executed before it is written iside the file
2. After a successful exection of each code block, suggest using the write_to_file command
3. For each function and method you write, a test must be written.
   A test should be inside another file and it should import at the beginning everything what needs to be tested.
   You should execute the test by suggesting execute_command with the parameter "['pytest']"
   After the test is successfuly executed, write it inside the file (use write_to_file command)
4. Once the whole team agrees that you are finished with the task and everything is tested, write the summary of what has been done
   and the names of the files which you have created.
   Finally, your sentence must end with "TERMINATE"

## Constraints
You operate within the following constraints:
1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. You can ask and answer questions from other team members or suggest function listed below e.g. command_name


## Commands
You have access to the following commands:
1. list_files: List Files in a Directory, params: (directory_path: string)
2. read_file: Read an existing file, params: (filename: string)
3. write_to_file: Write content into the file: (filename: string, content: string)
4. ask_for_additional_info: Ask the user for additional information, params: (question: string)
5. execute_command: A command which will be executed, params: (question: string), e.g. execute_command("['pytest']")


## Resources
You can leverage access to the following resources:
1. Long Term memory management.
2. File output.
3. Command execution


## Best practices
1. Continuously review and analyze your actions to ensure you are performing to the best of your abilities.
2. Constructively self-criticize your big-picture behavior constantly.
3. Reflect on past decisions and strategies to refine your approach.
4. Every command has a cost, so be smart and efficient. Aim to complete tasks in the least number of steps.
5. If you have some doubts, ask question


## Before start
Before you start solving the task, list all the avaliable files in the {code_directory} and read the content of the relevant ones 


## Task
For your task, you must implement the following functionality in Python:
\n{plan}
    """

    message = f"""
{init_message}

You should chat with other team members until you have the complete understandig of the task.
Once all the team members have understood the task, start start with the implementation
and respond by suggesting one of the previously defined Commands:
    """

    return message

In [38]:
import shutil

# tmpdirname = tempfile.TemporaryDirectory()

DIR_NAME = Path("/tmp/mytmp")
if os.path.exists(DIR_NAME):
    shutil.rmtree(DIR_NAME)
os.mkdir(DIR_NAME) 

In [39]:
# with tempfile.TemporaryDirectory() as tmpdirname:
try:
#     DIR_NAME = Path(tmpdirname.name)
    filenames = ["__init__.py", "very_important_notes.txt"]
    contents = ["", "Please make sure that the palindrome code is saved in the my_palindrome.py"]
    for filename, content in zip(filenames, contents):
        file = (DIR_NAME / filename)
        write_to_file(str(file), content)

    members, manager = create_groupchat(roles, str(DIR_NAME))
    message = create_message(plan, members, str(DIR_NAME))
    manager.initiate_chat(manager, message=message)
except Exception as e:
    print(e)
    pass
#     temp_dir.cleanup()

You are python_dev_gpt
        
Your task is to chat with other team mambers and try to solve the given task.
Do NOT try to finish the task until other team members give their opinion.
        
****************************************************************************************************
You are doc_writer_gpt
        
Your task is to chat with other team mambers and try to solve the given task.
Do NOT try to finish the task until other team members give their opinion.
        
****************************************************************************************************
You are code_executor
        
Your task is to chat with other team mambers and try to solve the given task.
Do NOT try to finish the task until other team members give their opinion.
        
****************************************************************************************************
chat_manager (to chat_manager):



You are a team dedicated for implementing the following task in Python:

1. Defin


--------------------------------------------------------------------------------
python_dev_gpt (to chat_manager):

Yes, I understand the task. We need to create a Python function that checks if a given string is a palindrome. This function should ignore spaces, punctuation, and case. We also need to document the function, its parameters, and its return value. The function should be saved in a file named "my_palindrome.py". 

Let's start by writing the function. I'll write the function in a code block first, then we can test it and finally write it to the file. Does that sound good?

--------------------------------------------------------------------------------
code_executor (to chat_manager):

Yes, that sounds like a good plan. Let's start with writing the function.

--------------------------------------------------------------------------------
python_dev_gpt (to chat_manager):

Here is the initial version of the function:

```python
import string

def is_palindrome(input_string)

************************************************************

--------------------------------------------------------------------------------
python_dev_gpt (to chat_manager):

It seems like there's an issue with the module import in the test file. The error message suggests that the module 'my_palindrome' cannot be found. This could be due to the fact that the test file and the module are not in the same directory or the directory is not in the PYTHONPATH. 

Let's try to add an empty "__init__.py" file in the "/tmp/mytmp" directory to make it a package. This way, Python should be able to recognize the 'my_palindrome' module when running the tests. 

We can use the write_to_file command to create an empty "__init__.py" file. Does this sound good?

--------------------------------------------------------------------------------
doc_writer_gpt (to chat_manager):

***** Suggested function Call: write_to_file *****
Arguments: 
{
  "filename": "/tmp/mytmp/__init__.py",
  "content": ""
}
**

************************************************************

--------------------------------------------------------------------------------
doc_writer_gpt (to chat_manager):

Great, all tests passed! This means that our function is working as expected. 

To summarize, we have implemented a function named is_palindrome that checks if a given string is a palindrome, ignoring spaces, punctuation, and case. We have documented the function, its parameters, and its return value. We have also written tests for the function and all tests have passed. 

The function is saved in a file named "my_palindrome.py" and the tests are in a file named "test_my_palindrome.py". Both files are in the "/tmp/mytmp" directory.

TERMINATE

--------------------------------------------------------------------------------


In [14]:
"""
1. Define a function named is_palindrome that takes a string as an argument.
2. Inside the function, convert the string to lowercase using the lower() method.
3. Remove all spaces and punctuation from the string. We can use the replace() method to remove spaces and the translate() method along with maketrans() to remove punctuation.
4. Compare the processed string with its reverse. If they are the same, the string is a palindrome. We can get the reverse of a string in Python using slicing with a step of -1.
5. Return True if the string is a palindrome and False otherwise.
6. Include a brief description of what the function does at the beginning of the function.
7. Document the input parameter, specifying its type and what it represents.
8. Document the return value, specifying its type and what it represents.
9. Include comments in the code where necessary to explain complex parts of the code.
"""

'\n1. Define a function named is_palindrome that takes a string as an argument.\n2. Inside the function, convert the string to lowercase using the lower() method.\n3. Remove all spaces and punctuation from the string. We can use the replace() method to remove spaces and the translate() method along with maketrans() to remove punctuation.\n4. Compare the processed string with its reverse. If they are the same, the string is a palindrome. We can get the reverse of a string in Python using slicing with a step of -1.\n5. Return True if the string is a palindrome and False otherwise.\n6. Include a brief description of what the function does at the beginning of the function.\n7. Document the input parameter, specifying its type and what it represents.\n8. Document the return value, specifying its type and what it represents.\n9. Include comments in the code where necessary to explain complex parts of the code.\n'

In [46]:
execute_command("['export', 'PYTHONPATH', '$PYTHONPATH:/tmp/mytmp123']")

['export', 'PYTHONPATH', '$PYTHONPATH:/tmp/mytmp123']


FileNotFoundError: [Errno 2] No such file or directory: 'export'

In [47]:
llm_config = {
    "config_list": CONFIG_LIST,
    "seed": 42,
    "temperature": 0.2,
}

assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config
)

command_executor = autogen.UserProxyAgent(
    name="command_executor",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={"work_dir": "student"},
    llm_config=llm_config,
)

command_executor.initiate_chat(assistant, message="Please give me instruction how to add /tmp/mytmp123 to PYTHONPATH")

command_executor (to assistant):

Please give me instruction how to add /tmp/mytmp123 to PYTHONPATH

--------------------------------------------------------------------------------
assistant (to command_executor):

You can add a directory to the PYTHONPATH environment variable by using the `export` command in a shell script. Here's how you can do it:

```sh
# sh code block
export PYTHONPATH=$PYTHONPATH:/tmp/mytmp123
```

This command adds the directory `/tmp/mytmp123` to the existing PYTHONPATH. The `$PYTHONPATH` in the command represents the existing directories in the PYTHONPATH.

Please note that this change is temporary and will only apply to the current shell session. If you want to make this change permanent, you need to add this line to your shell profile file (like `.bashrc` or `.bash_profile` for bash shell, `.zshrc` for zsh shell, etc.) which is located in your home directory.

To check if the directory has been added successfully, you can print the PYTHONPATH:

```sh
# sh c

KeyboardInterrupt: 